In [1]:
print("hello")

hello


In [2]:
%pwd

'/Users/amananand/PycharmProjects/Medical Chatbot/research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'/Users/amananand/PycharmProjects/Medical Chatbot'

In [10]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
#Extract the Data from pdf file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    document = loader.load()
    return document
    

In [12]:
extracted_data = load_pdf_file("Data/")

In [14]:
# extracted_data

In [31]:
# Split the data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200,chunk_overlap=200)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [32]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

19174


In [33]:
print(text_chunks[0])

page_content='The GALE
ENCYCLOPEDIA of
MEDICINE
THIRD EDITION' metadata={'source': 'Data/medical.pdf', 'page': 1}


In [34]:
print(text_chunks[1000])

page_content='enzyme is not very helpful in the evaluation of hyper-
tension (high blood pressure).
Preparation
Determination of ACE levels requires a blood
sample. The patient need not befasting.
Risks
Risks for this test are minimal, but may include
slight bleeding from the puncture site, fainting or
feeling lightheaded after venipuncture, or hematoma
(blood accumulating under the puncture site).
Normal results
Normal ranges for this test are laboratory-specific
but can range from 8-57 U/ml for patients over
20 years of age.
Abnormal results
Serum ACE levels are elevated in approximately
80-90% of patients with active sarcoidosis. Thyroid hor-
mone may have an effect on ACE activity, as hypothyr-
oid (low thyroid) patients, as well as patients with
anorexia nervosawithassociatedfindingsof hypothyroid-
ism, may have low serum ACE activity. ACE can also be
decreased in lungcancer(bronchogenic carcinoma).
Resources
BOOKS
Pagana, Kathleen Deska.Mosby’s Manual of Diagnostic and
Laboratory

In [36]:
# text_chunks

In [56]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from sentence_transformers import SentenceTransformer


In [57]:

# Download the Embeddings from the Hugging face
def download_huggingface_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MIniLM-L6-v2")
    return embeddings

In [58]:
embedding_model = download_huggingface_embeddings()

In [59]:
query_embeddings = embedding_model.embed_query("Hello World")
print(len(query_embeddings))

384


In [60]:
query_embeddings

[-0.010300836525857449,
 0.18307927250862122,
 0.030811265110969543,
 0.004452916327863932,
 -0.02733616530895233,
 -0.033562563359737396,
 0.03763153403997421,
 -0.031573306769132614,
 -0.0033910474739968777,
 -0.00895080715417862,
 0.03803619369864464,
 -0.05129102244973183,
 0.0003683085087686777,
 -0.02372707612812519,
 0.09271018952131271,
 -0.027795834466814995,
 -0.035152602940797806,
 -0.003224237821996212,
 -0.0768178403377533,
 -0.05761202797293663,
 0.07257590442895889,
 0.11128543317317963,
 0.01605853997170925,
 0.015908462926745415,
 -0.08232703804969788,
 0.00700724869966507,
 0.029013091698288918,
 0.0011386971455067396,
 0.1167173758149147,
 -0.03232740983366966,
 -0.032271649688482285,
 -0.001259012147784233,
 0.10591625422239304,
 0.02360082045197487,
 0.009664948098361492,
 0.09834086149930954,
 0.04293639957904816,
 -0.01954767107963562,
 0.019267868250608444,
 -0.06417101621627808,
 0.023923484608530998,
 -0.05288001894950867,
 -0.026469511911273003,
 0.0055487649

In [78]:
from dotenv import load_dotenv
load_dotenv()

True

In [80]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [64]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os 

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-bot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    )
)

In [81]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [68]:
from langchain_community.vectorstores import Pinecone

In [69]:
docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embedding_model,
)

In [70]:
#existing index
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embedding_model,
)

In [71]:
docsearch

In [74]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [75]:
retrieved_docs = retriever.invoke("What is acne?")

In [76]:
retrieved_docs

[Document(metadata={'page': 3353.0, 'source': 'Data/medical.pdf'}, page_content='(Custom Medical Stock Photo. Reproduced by permission.)\nKEY TERMS\nAcne— A chronic inflammation of the sebaceous\nglands that manifests as blackheads, whiteheads,\nand/or pustules on the face or trunk.\nPsoriasis— A skin disorder of chronic, itchy scaling\nmost commonly at sites of repeated minor trauma\n(e.g. elbows, knees, and skin folds). It affects up to\n2% of the population in Western countries—males\nand females equally.\nRosacea— A chronic inflammation of the face, with\nassociated scattered round nodules and increased\nreactivity of the facial capillaries to heat. It is most\ncommon in females, aged 30–50 years.\n3324 GALE ENCYCLOPEDIA OF MEDICINE\nSeborrheic dermatitis'),
 Document(metadata={'page': 54.0, 'source': 'Data/medical.pdf'}, page_content='Pathological Stage and Recurrence in Radical\nProstatectomy Cases.’’Journal of Urology (March\n1998): 935-940.\nNancy J. Nordenson\nAcid reflux see 

In [114]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4)

In [115]:
from langchain.chains import create_retrieval_chain

In [116]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [138]:
system_prompt = """You are a strict context-based question-answering assistant.
IMPORTANT RULES:
- ONLY use the information provided in the context below to answer questions
- If the context doesn't contain information to answer the question fully, respond with "I cannot answer this question based on the provided context."
- Never use your general knowledge to supplement answers
- Never make assumptions or inferences beyond what's explicitly stated in the context
- If only partial information is available, specify that you can only answer that part
- Keep answers to three sentences maximum and be concise

Context for your answers:
{context}

Remember: If you're unsure or if the context doesn't contain the relevant information, respond with "I cannot answer this question based on the provided context."
"""

prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [139]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [142]:
response = rag_chain.invoke({"input":"What is stats?"})
print(response["answer"])


System: I cannot answer this question based on the provided context.
